In [7]:

import tkinter as tk
from tkinter import filedialog, messagebox, ttk, simpledialog
import numpy as np
import pandas as pd
import SimpleITK as sitk
import import_ipynb
from CORE import Mask


In [ ]:
class Manager(tk.Tk):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.df_features = []
        self.source_folder = tk.StringVar()
        self.mask = Mask(self.source_folder.get())
        self.title("Feature Extraction")
        self.geometry("600x600")
        self.mode = tk.StringVar()
        self.roi = tk.StringVar()
        container = tk.Frame(self)
        container.pack(side="top", fill="both", expand=True)
        
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}

        for F in (Home, Processing, ROI, Masks, Features):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        
        self.show_frame(Home)
        self.protocol("WM_DELETE_WINDOW", self.on_closing)  # Handle window close button (X)
    def show_frame(self, container):
        frame = self.frames[container]
        frame.tkraise()
    def on_closing(self):
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            self.destroy()
            self.quit()

class Home(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.grid(row=0, column=0, sticky="nsew")
        self.create_widgets()

    def move_to_extraction(self, mode):
        self.controller.mode.set(mode)
        self.controller.show_frame(Processing)

    def create_widgets(self):
        for i in range(5): 
            self.grid_rowconfigure(i, weight=1)
            self.grid_columnconfigure(i, weight=1)
      
        tk.Button(self, text='Radiomics', command=lambda: self.move_to_extraction('Radiomics')).grid(row=2, column=1, padx=10, pady=10)
        tk.Button(self, text='Dosiomics', command=lambda: self.move_to_extraction('Dosiomics')).grid(row=2, column=2, padx=10, pady=10)
        tk.Button(self, text='Radiomics and Dosiomics', command=lambda: self.move_to_extraction('Radiomics and Dosiomics')).grid(row=2, column=3, padx=10, pady=10)

class Processing(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.grid(row=0, column=0, sticky="nsew")
        self.roi_var = tk.StringVar()
        self.selected_rois = []
        self.create_widgets()

    def create_widgets(self):
        top_frame = tk.Frame(self)
        top_frame.pack(expand=True)

        tk.Label(top_frame, text='Select folder:').pack(side='left', pady=10)
        tk.Entry(top_frame, textvariable=self.controller.source_folder, state='readonly', width=30).pack(side='left', pady=10)
        tk.Button(top_frame, text='Select', command=self.select_folder).pack(side='left', pady=10)

        bottom_frame = tk.Frame(self)
        bottom_frame.pack(expand=True)

        tk.Button(bottom_frame, text='Back', command=lambda: self.controller.show_frame(Home)).pack(side='left', pady=10)
        tk.Button(bottom_frame, text='Next', command=self.start_feature_extraction).pack(side='left', pady=10)

    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.controller.source_folder.set(folder)

    def start_feature_extraction(self):
        self.controller.mask = Mask(self.controller.source_folder.get())
        processing_label = tk.Label(self, text="Processing DICOM files ...")
        processing_label.pack(side='bottom', pady=10)
        self.controller.mask.path()
        self.after(100, lambda: self.process_cases(processing_label))

    def process_cases(self, label):
        self.controller.mask.process_cases()
        label.destroy()
        first_case_key = list(self.controller.mask.cases.keys())[0]
        if first_case_key:
            print(self.controller.mask.cases)
            print(f"First case key: {first_case_key}")
        self.controller.frames[ROI].first_case = first_case_key
        roi_names = self.controller.mask.select_first_case(first_case_key)
        if roi_names:
            self.controller.frames[ROI].show_rois()
        else:
            print(f"No ROI names found for case {first_case_key}.")
        self.controller.show_frame(ROI)

class ROI(tk.Frame):
   
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.grid(row=0, column=0, sticky="nsew")
        self.first_case = None
        self.progress_mask = tk.StringVar()
        self.current_case_key = None  
        self.case_iterator = None  
        self.create_widgets()

    def create_widgets(self):
        self.frame = tk.Frame(self)
        self.frame.pack(expand=True, fill="both")
        self.label = tk.Label(self.frame, text="Select ROI")
        self.label.pack()
        self.label_explication = tk.Label(self.frame, text="Select the ROI that will be used to generate the masks.")
        self.label_explication.pack()
        self.listbox = tk.Listbox(self.frame, selectmode=tk.SINGLE)
        self.listbox.pack(expand=True, fill=tk.BOTH)
        
        self.button_selection = tk.Button(self.frame, text="Select ROI", command=lambda: self.roi_selection('Only'))
        self.button_selection.pack(side='bottom', anchor='center', pady=10)
        self.button_all = tk.Button(self.frame, text="Apply selection to all cases", command=lambda: self.roi_selection('All'))
        self.button_all.pack(side='bottom', anchor='center', pady=10)

    def roi_selection(self, roi):
       
        selected_roi = self.listbox.curselection()
        if selected_roi:
            self.controller.roi.set(self.listbox.get(selected_roi))
            if roi == 'All':
                self.apply_to_all_cases()

            elif roi == 'Only':
                self.cases = list(self.controller.mask.cases.keys())
                self.case_iterator = iter(self.cases)
                self.process_next_case()

            self.listbox.destroy()
            self.label.destroy()
            self.button_selection.destroy()
            self.button_all.destroy()
    def process_next_case(self):
        try:
            case = next(self.case_iterator)
            self.current_case_key = case
            if 'ROI names' not in self.controller.mask.cases[case] or not self.controller.mask.cases[case]['ROI names']:
                messagebox.showinfo("Info", f"No ROIs available for case {case}. Skipping to next.")
                self.process_next_case()
            else:
                self.fill_listbox(case)
        except StopIteration:
            self.after(100, lambda: self.check_ROIs())

    def fill_listbox(self, case):
        if hasattr(self, 'listbox_frame'):
            self.listbox_frame.destroy()
        
        self.listbox_frame = tk.Frame(self.frame)
        self.listbox_frame.pack(expand=True, fill=tk.BOTH)
        
        self.case_label = tk.Label(self.listbox_frame, text=f'Select ROI for case {case}')
        self.case_label.pack(side='top', pady=5)
        
        self.listbox_not_matching = tk.Listbox(self.listbox_frame, selectmode=tk.SINGLE)
        self.listbox_not_matching.pack(expand=True, fill=tk.BOTH)

        self.button_select = tk.Button(self.listbox_frame, text='Select', command=self.select_roi)
        self.button_select.pack(side='bottom', pady=10)

        self.listbox_not_matching.delete(0, tk.END)  
        
        for roi_name in self.controller.mask.cases[case]['ROI names']:
            self.listbox_not_matching.insert(tk.END, roi_name)
        
        self.listbox_frame.after(100, self.set_listbox_focus)
    
    def set_listbox_focus(self):
        self.listbox_not_matching.focus_set()

    def select_roi(self):
        selected_roi = self.listbox_not_matching.curselection()
        if selected_roi:
            roi_name = self.listbox_not_matching.get(selected_roi)
            self.controller.mask.set_roi(self.current_case_key, roi_name)
            self.process_next_case()
        
    def apply_to_all_cases(self):
        if self.controller.roi:
            roi = self.controller.roi.get()
            _, self.cases_not_matching = self.controller.mask.find_similar_ROIs(roi)
        
        if self.cases_not_matching:
            messagebox.showinfo('Case not found', f'Case(s) {", ".join(self.cases_not_matching)} not found')
            self.case_iterator = iter(self.cases_not_matching)
            self.process_next_case()
        else:
            messagebox.showinfo('Done', 'All ROIs selected')
        self.after(100, lambda: self.check_ROIs())

    def start_mask(self):
        self.controller.show_frame(Masks)
        processing_label = tk.Label(self, text = "Generating masks ...")
        processing_label.pack(side='bottom', pady=10)
        self.after(100, lambda: self.create_mask(processing_label))
        
    def create_mask(self, label):
        label.destroy() 
        self.controller.frames[Masks].create_mask()

    def show_rois(self):
        if self.first_case is not None:
            rois = self.controller.mask.select_first_case(self.first_case)
            self.listbox.delete(0, tk.END)
            for roi in rois:
                self.listbox.insert(tk.END, roi)
    
    def check_ROIs(self):
        if hasattr(self, 'listbox_frame'):
            self.listbox_frame.destroy()

        self.listbox_frame = tk.Frame(self.frame)
        self.listbox_frame.pack(expand=True, fill=tk.BOTH)

        self.check_label = tk.Label(self.listbox_frame, text='Check ROIs')
        self.check_label.pack(side='top', pady=5)

        self.check_listbox = tk.Listbox(self.listbox_frame, selectmode=tk.MULTIPLE)
        self.check_listbox.pack(expand=True, fill=tk.BOTH)

        for key, value in self.controller.mask.cases.items():
            display_text = f"{key} - ROI: {value.get('Selected ROI', 'Not Selected')}"
            self.check_listbox.insert(tk.END, display_text)

        self.correct_button = tk.Button(self.listbox_frame, text='All ROIs are correct', command= self.start_mask)
        self.correct_button.pack(side='bottom', pady=10)

        self.incorrect_button = tk.Button(self.listbox_frame, text='Correct selected ROIs', command= self.correct_errors)
        self.incorrect_button.pack(side='bottom', pady=10)

        self.listbox_frame.after(100, self.check_listbox.focus_set())

    def correct_errors(self):
        selected_items = self.check_listbox.curselection()
        if selected_items:
            self.errors = [self.check_listbox.get(item) for item in selected_items]
            self.selected_keys = [item.split(" - ROI: ")[0] for item in self.errors]
            self.case_iterator = iter(self.selected_keys)
            self.process_next_case()

class Masks(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.grid(row=0, column=0, sticky="nsew")
        self.create_widgets()

    def create_widgets(self):
        self.mask_frame = tk.Frame(self)
        self.mask_frame.pack(expand=True)

        processing_label = tk.Label(self.mask_frame, text="Generating masks ...")
        processing_label.pack(side='top', pady=10)

        self.progress_mask = ttk.Progressbar(self.mask_frame, orient="horizontal", length=300, mode="determinate")
        self.progress_mask.pack(side='bottom', pady=10)

        self.button = tk.Button(self.mask_frame, text='Next' , command=self.start_extraction)
        self.button.pack(side='bottom', pady=10)

    def create_mask(self):
        if self.controller.mask.cases:
            
            total_cases = len(self.controller.mask.cases)
            self.progress_mask["maximum"] = total_cases
            self.progress_mask["value"] = 0
            for i, (key, value) in enumerate (self.controller.mask.cases.items(),1):
                if 'Selected ROI' in value: 
                    roi = value['Selected ROI']
                    self.controller.mask.create_binary_mask(key, roi)
         
                    
                self.progress_mask["value"] = i
                self.update_idletasks()
            messagebox.showinfo('Done', 'All masks created')
    
    def start_extraction(self):
        self.controller.show_frame(Features)
        self.after(100, lambda: self.extract_features())

    def extract_features(self):
        self.controller.frames[Features].feature_extraction() # cambiar 

class Features(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller
        self.grid(row=0, column=0, sticky="nsew")
        self.create_widgets()
        
    def create_widgets(self):
        self.features_frame = tk.Frame(self)
        self.features_frame.pack(expand=True)

        self.processing_label = tk.Label(self.features_frame, text="Feature extraction in progress ...", wraplength=400)
        self.processing_label.pack(side='top', pady=10)

        self.progress_features = ttk.Progressbar(self.features_frame, orient="horizontal", length=300, mode="determinate")
        self.progress_features.pack(side='bottom', pady=10)

        self.button = tk.Button(self.features_frame, text='Finish', command = self.controller.on_closing)
        self.button.pack(side='bottom', pady=10)
    
    def update_label(self, text):
        self.processing_label.config(text=text)
        self.update_idletasks()

    def on_closing(self):
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            self.destroy()  
            self.quit()  

    def feature_extraction(self):
        if self.controller.mode.get() in ['Radiomics', 'Dosiomics', 'Radiomics and Dosiomics']:
            self.update_label(" 'Bin count' refers to the number of intervals that span the full dose range in the image.")
            bin_count = simpledialog.askinteger("Input", "Please enter bin count:", minvalue=30, maxvalue=130)
            if bin_count is None:
                bin_count = 100
                messagebox.showinfo("Info", "No bin count entered. Using default value of 100.")

            eqd2 = False
            if self.controller.mode.get() in ['Dosiomics', 'Radiomics and Dosiomics']:
                self.update_label(" 'EQD2', or Equivalent Dose in 2 Gy fractions, simplifies the comparison of various radiation treatment schedules by converting them into a dose equivalent to what would be delivered in standard 2 Gy fractions.")
                eqd2 = messagebox.askyesno("EQD2", "Do you want to apply EQD2?")
                if eqd2:
                    alfa_beta = simpledialog.askinteger("Input", "Please enter alfa/beta ratio:")
                    if alfa_beta is None:
                        messagebox.showinfo("Info", "No alpha/beta ratio entered. Aborting EQD2 application.")
                        return

                resolution = None
                while True:
                    self.update_label("The dosimetric images and the masks must be rescaled to a similar size to extract the dosimetric features.")
                    resolution_str = simpledialog.askstring("Input", "Please enter new resolution in the format: 'x, y, z'")
                    if resolution_str is None:
                        resolution = (1.0, 1.0, 1.0)
                        messagebox.showinfo("Info", "No resolution entered. Using default value of (1.0, 1.0, 1.0).")
                        break

                    resolution_parts = resolution_str.split(',')
                    if len(resolution_parts) != 3:
                        messagebox.showwarning("Warning", "Invalid input format. Please enter three values separated by commas.")
                        continue
                    try:
                        resolution = tuple(map(float, resolution_parts))
                        break
                    except ValueError:
                        messagebox.showwarning("Warning", "Invalid input format. Please enter numeric values separated by commas.")

            self.dfs = []
            total_cases = len(self.controller.mask.cases)
            self.progress_features["maximum"] = total_cases
            self.progress_features["value"] = 0

            for i, (key, value) in enumerate(self.controller.mask.cases.items(), 1):
                try:
                    if 'CT nifti' not in value or 'Mask path' not in value:
                        print(f"Skipping case {key} due to missing 'CT nifti' or 'Mask path'")
                        continue

                    if self.controller.mode.get() in ['Radiomics', 'Radiomics and Dosiomics']:
                        image_path_radiomic = value['CT nifti']
                        mask_path_radiomic = value['Mask path']
                        
                        self.update_label("Extracting radiomic features ...")
                        bin_width_radiomic = self.controller.mask.select_bin_width(image_path_radiomic, bin_count)
                        featureVector_radiomic = self.controller.mask.extract_features(mask_path_radiomic, image_path_radiomic, bin_width_radiomic)
                        df_radiomic = self.controller.mask.dataframe_features(featureVector_radiomic, key)
                        df_radiomic = df_radiomic.applymap(lambda x: float(x) if isinstance(x, np.generic) else x)
                        df_radiomic.columns = ['Radiomic_' + col for col in df_radiomic.columns]  
                        self.dfs.append(df_radiomic)

                    if self.controller.mode.get() in ['Dosiomics', 'Radiomics and Dosiomics']:
                        if 'RD nifti' not in value:
                            print(f"Skipping case {key} due to missing 'RD nifti'")
                            continue

                        image_path_dosiomic = value['RD nifti']
                        mask_path_dosiomic = value['Mask path']
                        output_filename_mask = f"Mask_rescaled_{key}.nii"
                        output_filename_rd = f"RD_rescaled_{key}.nii"
                        self.update_label("Resampling dosimetric image and mask ...")
                        image_rescaled = self.controller.mask.resample(image_path_dosiomic, resolution, sitk.sitkBSpline, key, True)
                        mask_rescaled = self.controller.mask.resample(mask_path_dosiomic, resolution, sitk.sitkNearestNeighbor, key)
                        value['Mask rescaled path'] = self.controller.mask.save_nifti_image(mask_rescaled, key, output_filename_mask)

                        if eqd2:
                            try:
                                eqd2_image = self.controller.mask.EQD2(alfa_beta, key, image_rescaled)
                                value['RD rescaled path'] = self.controller.mask.save_nifti_image(eqd2_image, key, output_filename_rd)
                            except ValueError as e:
                                if "Please enter the number of sessions" in str(e):
                                    sessions = simpledialog.askinteger("Input", f"Please enter the number of sessions for case {key}:")
                                    if sessions is not None:
                                        eqd2_image = self.controller.mask.EQD2(alfa_beta, key, image_rescaled, sessions)
                                        value['RD rescaled path'] = self.controller.mask.save_nifti_image(eqd2_image, key, output_filename_rd)
                                    else:
                                        messagebox.showinfo("Info", f"No sessions entered for case {key}. Skipping case.")
                                        continue
                                else:
                                    print(f"Skipping case {key} due to ValueError in EQD2: {e}")
                                    continue
                        else:
                            value['RD rescaled path'] = self.controller.mask.save_nifti_image(image_rescaled, key, output_filename_rd)

                        image_path_dosiomic = value['RD rescaled path']
                        mask_path_dosiomic = value['Mask rescaled path']

                        self.update_label("Extracting dosiomic features ...")
                        bin_width_dosiomic = self.controller.mask.select_bin_width(image_path_dosiomic, bin_count)
                        featureVector_dosiomic = self.controller.mask.extract_features(mask_path_dosiomic, image_path_dosiomic, bin_width_dosiomic)
                        df_dosiomic = self.controller.mask.dataframe_features(featureVector_dosiomic, key)
                        df_dosiomic = df_dosiomic.applymap(lambda x: float(x) if isinstance(x, np.generic) else x)
                        df_dosiomic.columns = ['Dosiomic_' + col for col in df_dosiomic.columns]  
                        self.dfs.append(df_dosiomic)

                    self.progress_features["value"] = i 
                    self.update_idletasks()

                except KeyError as e:
                    print(f"Skipping case {key} due to missing key: {e}")
                    continue
                except Exception as e:
                    print(f"Error processing case {key}: {e}")
                    continue

            if self.dfs:
                combined_df = pd.concat(self.dfs, axis=1, join='outer').T.drop_duplicates().T
                self.controller.df_features = combined_df
    
                messagebox.showinfo('Done', 'features extracted')
            else:
                messagebox.showinfo('Error', 'No dataframes to combine')

            df_extract = messagebox.askyesno("Save", "Do you want to save the extracted features?")
            if df_extract:
                if self.controller.mode.get() == 'Radiomics':
                    self.controller.mask.df_to_excel(self.controller.df_features, 'Radiomics.xlsx')
                    print("Extracted features saved to Radiomics.xlsx.")
                elif self.controller.mode.get() == 'Dosiomics':
                    self.controller.mask.df_to_excel(self.controller.df_features, 'Dosiomics.xlsx')
                    print("Extracted features saved to Dosiomics.xlsx.")
                elif self.controller.mode.get() == 'Radiomics and Dosiomics':
                    self.controller.mask.df_to_excel(self.controller.df_features, 'Radiomics_Dosiomics.xlsx')
                    print("Extracted features saved to Radiomics_Dosiomics.xlsx.")
  
if __name__=='__main__':
    app = Manager()
    app.mainloop()
    print(app.df_features)